In [47]:
import torch
import torch.nn.functional as F
import torch.nn as nn




In [45]:
# We’ll represent the input, a sequence of t vectors of dimension k as a t by k matrix 𝐗.
# Including a minibatch dimension b, gives us an input tensor of size (b,t,k).
# assume we have some tensor x with size (b, t, k)
x=torch.randn((32,3,2))
raw_weights = torch.bmm(x, x.transpose(1, 2))

# print(f"Raw weights shape {raw_weights.shape}\n")
# print(x[:1,])
# print(raw_weights[:1,])

weights=torch.softmax(raw_weights,dim=2)

y = torch.bmm(weights, x)

In [48]:
class SelfAttention(nn.Module):
    def __init__(self, k, heads=4, mask=False):
        super(SelfAttention).__init__()
    
        assert k % heads == 0
    
        self.k, self.heads = k, heads
        
        # These compute the queries, keys and values for all
        # heads
        self.tokeys    = nn.Linear(k, k, bias=False)
        self.toqueries = nn.Linear(k, k, bias=False)
        self.tovalues  = nn.Linear(k, k, bias=False)

	    # This will be applied after the multi-head self-attention operation.
        self.unifyheads = nn.Linear(k, k)

        
    def forward(self, x):

        b, t, k = x.size()
        h = self.heads

        queries = self.toqueries(x)
        keys    = self.tokeys(x)   
        values  = self.tovalues(x)
    	s = k // h

	    keys    = keys.view(b, t, h, s)
	    queries = queries.view(b, t, h, s)
    	values  = values.view(b, t, h, s)
